In [93]:
# 20yy년 데이터 작업<<<<<<<<
# 20yy년 데이터 작업<<<<<<<<
# 20yy년 데이터 작업<<<<<<<<
yyyy = '2011'     #<<<<<<<<<<<<<<< 작업데이터 년도를 내가 입력해야됨!!!!!
# 20yy년 데이터 작업<<<<<<<<
# 20yy년 데이터 작업<<<<<<<<
# 20yy년 데이터 작업<<<<<<<<
# 2022-11-07 20:00 월요일 : 기존 데이터셋의 특보강도를 특보없음0 특보있음1 이렇게 이진분류용으로 수정함!


## 코랩을 사용할 때
#드라이브 마운트
from google.colab import drive
drive.mount('/content/drive/')

#현재 작업 위치 이동
#띄어쓰기에 \붙일 것
%cd /content/drive/MyDrive/Dev_work/REPORT



import pandas as pd
import numpy as np
from datetime import timedelta
import datetime as dt

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/Dev_work/REPORT


In [94]:
# /content/drive/MyDrive/Dev_work/REPORT/DATA_csv/전체데이터_이진분류/BinaryClassi_2020_GEOJE.csv

In [95]:
# df = pd.read_csv(   '/content/drive/MyDrive/Dev_work/REPORT/DATA_csv/전체데이터_이진분류/BinaryClassi_' + yyyy +'_GEOJE.csv', index_col=False)
df = pd.read_csv(   '/content/drive/MyDrive/Dev_work/REPORT/DATA_csv/전체데이터_이진분류/BinaryClassi_' + yyyy +'.csv', index_col=False)
# df = df.set_index('', drop=True)
df

,일시,WIND_SPEED,GUST_WIND,SIG_WAVE,MAX_WAVE,HPA,특보강도
0,2011-03-04 17:00:00,2.8,8.9,0.3,0.5,1021.0,0.0
1,2011-03-04 18:00:00,3.4,8.9,0.3,0.4,1021.7,0.0
2,2011-03-04 19:00:00,4.5,8.9,0.3,0.4,1022.2,0.0
3,2011-03-04 20:00:00,2.9,8.9,0.3,0.5,1022.6,0.0
4,2011-03-04 21:00:00,3.8,8.9,0.3,0.5,1023.1,0.0
...,...,...,...,...,...,...,...
5587,2011-10-25 04:00:00,5.1,11.2,0.2,0.3,1015.9,0.0
5588,2011-10-25 05:00:00,6.4,11.2,0.2,0.3,1016.8,0.0
5589,2011-10-25 06:00:00,6.7,11.2,0.1,0.2,1017.5,0.0
5590,2011-10-25 07:00:00,4.4,11.2,0.2,0.3,1018.4,0.0


In [96]:
ndf = df
ndf #p211 보면 9월7일 am3시부터 특보레벨이 엄청 뛰어있음 

,일시,WIND_SPEED,GUST_WIND,SIG_WAVE,MAX_WAVE,HPA,특보강도
0,2011-03-04 17:00:00,2.8,8.9,0.3,0.5,1021.0,0.0
1,2011-03-04 18:00:00,3.4,8.9,0.3,0.4,1021.7,0.0
2,2011-03-04 19:00:00,4.5,8.9,0.3,0.4,1022.2,0.0
3,2011-03-04 20:00:00,2.9,8.9,0.3,0.5,1022.6,0.0
4,2011-03-04 21:00:00,3.8,8.9,0.3,0.5,1023.1,0.0
...,...,...,...,...,...,...,...
5587,2011-10-25 04:00:00,5.1,11.2,0.2,0.3,1015.9,0.0
5588,2011-10-25 05:00:00,6.4,11.2,0.2,0.3,1016.8,0.0
5589,2011-10-25 06:00:00,6.7,11.2,0.1,0.2,1017.5,0.0
5590,2011-10-25 07:00:00,4.4,11.2,0.2,0.3,1018.4,0.0


## 1107 15:00 자 지금부터 생각해보자: 특보강도1의 row수만큼 앞뒤시점을 뽑아낼 방법 
> * 풍속 파고등의 결측치 처리 때문에 2020-12-29 03:00 ~ 2020-12-31 15:00의 풍랑주의보는 뒤시점(특보강도=0)이 다 날아가서 쓸 수 없다.
> * 그럼 남은 "특보강도0~특보강도1~특보강도0"의 건수가 몇 건일까?
>> * 일단 특보강도0을 전부 null로 비우자. 그 DF의 이름은 nullDF
>> * if문으로, nullDF.iloc[i,6] != nullDF.iloc[i+1,6]: 일때,
>> * 그 [i,6]의 인덱스를 리스트 lv00_Index_list에 넣어라(append)
>> * 그리고 [i+1,6]의 인덱스는 리스트 lv01_Index_list에 넣어라 (append)
> * 아니다 일단 특보강도 1의 행수를 빼보면 되잖아.
> * 그리고 마지막 12월 30일 ~ 31일의 ndf['특보강도']=1은 전부 드롭해

In [97]:
# 이진분류를 위해 특보강도 2,3,4를 모두 1로 바꾸겠습니다

ndf['특보강도'] = ndf['특보강도'].replace(2,1)
ndf['특보강도'] = ndf['특보강도'].replace(3,1)
ndf['특보강도'] = ndf['특보강도'].replace(4,1)
ndf['특보강도']

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
5587    0.0
5588    0.0
5589    0.0
5590    0.0
5591    0.0
Name: 특보강도, Length: 5592, dtype: float64

In [98]:
ndf

,일시,WIND_SPEED,GUST_WIND,SIG_WAVE,MAX_WAVE,HPA,특보강도
0,2011-03-04 17:00:00,2.8,8.9,0.3,0.5,1021.0,0.0
1,2011-03-04 18:00:00,3.4,8.9,0.3,0.4,1021.7,0.0
2,2011-03-04 19:00:00,4.5,8.9,0.3,0.4,1022.2,0.0
3,2011-03-04 20:00:00,2.9,8.9,0.3,0.5,1022.6,0.0
4,2011-03-04 21:00:00,3.8,8.9,0.3,0.5,1023.1,0.0
...,...,...,...,...,...,...,...
5587,2011-10-25 04:00:00,5.1,11.2,0.2,0.3,1015.9,0.0
5588,2011-10-25 05:00:00,6.4,11.2,0.2,0.3,1016.8,0.0
5589,2011-10-25 06:00:00,6.7,11.2,0.1,0.2,1017.5,0.0
5590,2011-10-25 07:00:00,4.4,11.2,0.2,0.3,1018.4,0.0


In [99]:
ndf.tail()

,일시,WIND_SPEED,GUST_WIND,SIG_WAVE,MAX_WAVE,HPA,특보강도
5587,2011-10-25 04:00:00,5.1,11.2,0.2,0.3,1015.9,0.0
5588,2011-10-25 05:00:00,6.4,11.2,0.2,0.3,1016.8,0.0
5589,2011-10-25 06:00:00,6.7,11.2,0.1,0.2,1017.5,0.0
5590,2011-10-25 07:00:00,4.4,11.2,0.2,0.3,1018.4,0.0
5591,2011-10-25 08:00:00,4.0,11.2,0.2,0.4,1019.7,0.0


## 1107 20:00 헉 예은님 구세주 갓예은;
> * 투썸에서 논문 찾아보며 우리 모델1과 흡사한 케이스를 확인했다함.
> * 이진분류로 하면 되고, 논문의 모델도 우리가 한번씩 돌려봤던 거라고 함
> * 나는 일단 이진분류로 전처리를 하겠습니당

In [100]:
# ndf = ndf.to_csv(  '/content/drive/MyDrive/Dev_work/REPORT/DATA_csv/전체데이터_이진분류/이진분류after/BinaryClassi_'+ yyyy +'_Geoje_ver01.csv'  , index='False', encoding='UTF-8')
ndf = ndf.to_csv(  '/content/drive/MyDrive/Dev_work/REPORT/DATA_csv/전체데이터_이진분류/이진분류after/BinaryClassi_'+ yyyy +'_ver01.csv'  , index='False', encoding='UTF-8')